In [ ]:
import random
import time
import csv

In [ ]:
class NN:
    def __init__(self, hidden_activation, output_activation, epochs, learning_rate, random_seed):
        self.hidden_activation = hidden_activation
        self.output_activation = output_activation
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.random_seed = random_seed
        self.napier_number = self.napiers_logarithm(100000000)
        self.tolerance = 1e-10  # sqrt許容誤差
    
    def napiers_logarithm(self, x):
        """
        ネイピア数を求める関数

        Args:
            x (float): 入力
        
        Returns:
            float: 出力
        """
        return (1 + 1 / x) ** x
    
    def ln(self, x, max_iter=20, tol=1e-12):
        """
        自然対数を求める関数

        Args:
            x (float): 入力
            max_iter (int): 最大反復回数
            tol (float): 許容誤差
        
        Returns:
            float: 出力
        
        Raises:
            ValueError: x が正でない場合
        """
        if x <= 0: raise ValueError("x must be positive")
        k = 0
        while x > 2:
            x /= 2
            k += 1
        while x < 0.5:
            x *= 2
            k -= 1
        y = x - 1  # ln(1) = 0 付近の値から開始
        for _ in range(max_iter):
            prev_y = y
            y -= (2.718281828459045**y - x) / (2.718281828459045**y)  # f(y) / f'(y)
            if abs(y - prev_y) < tol:
                break
        return y + k * 0.6931471805599453  # ln(2) ≈ 0.693147
    
    def sqrt(self, x):
        """
        平方根を求める関数

        Args:
            x (float): 入力
        
        Returns:
            float: 出力
        """
        estimate = x / 2.0  # 初期推定値
        while True:
            new_estimate = (estimate + x / estimate) / 2  # ニュートン法による更新
            if abs(new_estimate - estimate) < self.tolerance:  # 収束したら終了
                return new_estimate
            estimate = new_estimate  # 推定値を更新


    def sigmoid(self, x, derivative=False):
        """
        Sigmoid 関数およびその微分

        Args:
            x (float): 入力
            derivative (bool): 微分を計算するかどうか
        
        Returns:
            float: 出力
        """
        if derivative:
            return self.sigmoid_derivative(x)
        return 1 / (1 + self.napier_number ** -x)

    def sigmoid_derivative(self, x):
        """
        Sigmoid 関数の微分

        Args:
            x (float): 入力
        
        Returns:
            float: 出力
        """
        return self.sigmoid(x) * (1 - self.sigmoid(x))


    def relu(self, x, derivative=False):
        """
        ReLU 関数およびその微分

        Args:
            x (float): 入力
            derivative (bool): 微分を計算するかどうか
        
        Returns:
            float: 出力
        """
        if derivative:
            return self.relu_derivative(x)
        return max(0, x)

    def relu_derivative(self, x):
        """
        ReLU 関数の微分

        Args:
            x (float): 入力
        
        Returns:
            float: 出力
        """
        return 1 if x > 0 else 0
    

    def leaky_relu(self, x, alpha=0.01, derivative=False):
        """
        Leaky ReLU 関数およびその微分

        Args:
            x (float): 入力
            alpha (float): ハイパーパラメータ
            derivative (bool): 微分を計算するかどうか
        
        Returns:
            float: 出力
        """
        if derivative:
            return self.leaky_relu_derivative(x, alpha)
        return x if x > 0 else alpha * x

    def leaky_relu_derivative(self, x, alpha=0.01):
        """
        Leaky ReLU 関数の微分

        Args:
            x (float): 入力
            alpha (float): ハイパーパラメータ
        
        Returns:
            float: 出力
        """
        return 1 if x > 0 else alpha
    

    def identity(self, x, derivative=False):
        """
        恒等関数およびその微分

        Args:
            x (float): 入力
            derivative (bool): 微分を計算するかどうか
        
        Returns:
            float: 出力
        """
        if derivative:
            return self.identity_derivative(x)
        return x

    def identity_derivative(self, x):
        """
        恒等関数の微分

        Args:
            x (float): 入力(未使用)
        
        Returns:
            int: 出力
        """
        return 1
    

    def cross_entropy_loss(self, y_true, y_pred):
        """
        交差エントロピー損失関数

        Args:
            y_true (list): 正解ラベル
            y_pred (list): 予測ラベル
        
        Returns:
            float: 出力
            
        Raises:
            ValueError: 入力リストの長さが異なる場合
        """
        if len(y_true) != len(y_pred): raise ValueError("Input lists must have the same length.")
        return -sum([t * self.ln(p + 1e-9) for t, p in zip(y_true, y_pred)])
    
    def mean_squared_error(self, y_true, y_pred):
        """
        平均二乗誤差を求める関数

        Args:
            y_true (list): 正解ラベル
            y_pred (list): 予測ラベル
        
        Returns:
            float: 出力
            
        Raises:
            ValueError: 入力リストの長さが異なる場合
        """
        if len(y_true) != len(y_pred): raise ValueError("Input lists must have the same length.")
        return sum([(t - p) ** 2 for t, p in zip(y_true, y_pred)]) / len(y_true)
    
    def mean_absolute_error(self, y_true, y_pred):
        """
        平均絶対誤差を求める関数

        Args:
            y_true (list): 正解ラベル
            y_pred (list): 予測ラベル
        
        Returns:
            float: 出力
        
        Raises:
            ValueError: 入力リストの長さが異なる場合
        """
        if len(y_true) != len(y_pred): raise ValueError("Input lists must have the same length.")
        return sum([abs(t - p) for t, p in zip(y_true, y_pred)]) / len(y_true)
    
    def binary_cross_entropy_loss(self, y_true, y_pred):
        """
        バイナリ交差エントロピー損失関数

        Args:
            y_true (list): 正解ラベル
            y_pred (list): 予測ラベル
        
        Returns:
            float: 出力
        
        Raises:
            ValueError: 入力リストの長さが異なる場合
        """
        if len(y_true) != len(y_pred): raise ValueError("Input lists must have the same length.")
        epsilon = 1e-9  # 0で割るのを防ぐための小さな値
        return -sum([t * self.ln(p + epsilon) + (1 - t) * self.ln(1 - p + epsilon) for t, p in zip(y_true, y_pred)]) / len(y_true)
    
    def categorical_cross_entropy_loss(self, y_true, y_pred):
        """
        カテゴリカル交差エントロピー損失関数

        Args:
            y_true (list): 正解ラベル
            y_pred (list): 予測ラベル
        
        Returns:
            float: 出力
        
        Raises:
            ValueError: 入力リストの長さが異なる場合
        """
        if len(y_true) != len(y_pred): raise ValueError("Input lists must have the same length.")
        epsilon = 1e-9  # 0で割るのを防ぐための小さな値
        return -sum([t * self.ln(p + epsilon) for t, p in zip(y_true, y_pred)]) / len(y_true)
    
    def initialize_weights(self, layer_sizes):  # 重みとバイアスの初期化
        """
        重みとバイアスを初期化する関数

        Args:
            layer_sizes (list): 各層のユニット数
        
        Returns:
            tuple: 重みとバイアス
        """
        weights, biases = [], []
        for i in range(len(layer_sizes) - 1):
            limit = self.sqrt(6 / (layer_sizes[i] + layer_sizes[i+1]))  # 重みの初期化に使う乱数の範囲
            weights.append([[random.uniform(-limit, limit) for _ in range(layer_sizes[i])] for _ in range(layer_sizes[i+1])])  # 重みは -limit から limit の間の乱数で初期化
            biases.append([0 for _ in range(layer_sizes[i+1])])  # バイアスは0で初期化
        return weights, biases
    
    
    def fit(self, X, y):
        pass
